# OpenStreetMap Data Case Study

### Map Area

Brooklyn, New York, USA

Brooklyn is one of the five boroughs of New York City.

* https://www.openstreetmap.org/node/158857828#map=13/40.6502/-73.9497
* https://mapzen.com/data/metro-extracts/metro/brooklyn_new-york/

Brooklyn is my hometown. I'm curious to see the contributions to the map so far, how messy the data is, and what we can reveal about the city via query. Also, I’d like to discuss an opportunity to contribute to its improvement on OpenStreetMap.org

In [9]:
import csv
import collections
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
from collections import defaultdict

In [10]:
brooklyn = "brooklyn_new-york.osm"  
sample = "sample.osm"


### Data Audit
I parse through the Brooklyn dataset with ElementTree and count the unique element type using the count_tags function.

In [11]:
def get_element(osm_file):
	context = ET.iterparse(osm_file, events=('start', 'end'))
	_, root = next(context)
	for event, elem in context:
		if event == 'end':
			yield elem
			root.clear()

In [12]:
def count_tags(osm_file):
    tags = {}
    for elem in get_element(osm_file):
        if elem.tag in tags.keys():
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
        elem.clear()
    return tags

brooklyn_tags = count_tags("sample.osm")
pprint.pprint(brooklyn_tags)

{'member': 439,
 'nd': 71496,
 'node': 49947,
 'osm': 1,
 'relation': 42,
 'tag': 56704,
 'way': 9864}


Next I check ‘k’ value for each tag. There are three regular expression, lower is for tags that contain only lowercase letters and are valid. lower_colon is for other valid tags with a colon in the value. problemchars is for tags with problematic characters. Finally I obtain a dictionary which contain the count of each of three tag categories above.

In [13]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if re.match(lower, element.attrib['k']):
            keys["lower"] += 1
        elif re.match(lower_colon, element.attrib['k']):
            keys["lower_colon"] += 1
        elif re.search(problemchars, element.attrib['k']):
            keys["problemchars"] += 1
        else:
            keys['other'] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

brooklyn_all_keys = process_map("sample.osm")
print brooklyn_all_keys

{'problemchars': 285, 'lower': 21426, 'other': 131, 'lower_colon': 34862}


## Problems Occured in the Map

After initially downloading a small sample size of the Brooklyn area and running it against a provisional data.py file, I noticed five main problems with the data, which I will discuss in the following order:
1. Abbreviated Street Names
2. Misspelled Street Names
3. "Incorrect" Postal Codes (Brooklyn area zip codes all begin with “112” however a large portion of all documented zip codes were outside this region).
4. "Incorrect" Phone Numbers (Brooklyn phone numbers all begin with  "718" but a portion of all phone numbers were outside this region).
5. Inconsistent Phone Numbers (+1-718-425-8769, 718) 235-0444,718-418-0793).

### Overabbreviated street names and Misspelled street names

The first problem I encountered in this dataset is from the street name abbreviation. So build the regular expression to match the last element in the string, where usually the street type is based. Then based on the street abbreviation, create a mapping that need to be cleaned.

Another problem in this dataset is misspelled street names like "steet" and "avene". Based on misspelled street names, create a mapping to that need to be corrected.


In [19]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Highway", "Parkway", "Road", "Extension",
           "Path", "Park", "Plaza", "Walk", "Square", "Piers", "Lane", "Center"]

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def audit_street(osmfile):
	osm_file = open(osmfile, "r")
	street_types = defaultdict(set)
	for i, elem in enumerate(get_element(osmfile)):
		if elem.tag in ["node" ,"way"]:
			for tag in elem.iter('tag'):
				if is_street_name(tag):
					audit_street_type(street_types, tag.attrib['v'])
	osm_file.close()
	return street_types
    
bk_street_types = audit_street(sample)
pprint.pprint(dict(bk_street_types))

{'A': set(['Avenue A']),
 'Alley': set(['Cortlandt Alley', 'Grace Court Alley', 'Mac Dougal Alley']),
 'Americas': set(['Avenue Of The Americas']),
 'B': set(['Avenue B']),
 'Bayside': set(['Bayside']),
 'Bowery': set(['Bowery']),
 'Broadway': set(['Broadway', 'East Broadway', 'West Broadway']),
 'C': set(['Avenue C']),
 'Crescent': set(['Boelsen Crescent',
                  'Cromwell Crescent',
                  'Dieterle Crescent',
                  'Ellwell Crescent',
                  'Slocum Crescent']),
 'D': set(['Avenue D']),
 'East': set(['Cadman Plaza East',
              'Gramercy Park East',
              'Village Road East',
              'Williamsburg Street East']),
 'Expressway': set(['Horace Harding Expressway']),
 'F': set(['Avenue F']),
 'H': set(['Avenue H']),
 'Hamilton': set(['Fort Hamilton']),
 'Heights': set(['Columbia Heights']),
 'I': set(['Avenue I']),
 'J': set(['Avenue J']),
 'K': set(['Avenue K']),
 'L': set(['Avenue L']),
 'M': set(['Avenue M']),
 'Mews':

In [20]:
mapping = { "Ave":"Avenue",
            "Ave.":"Avenue",
            "ave":"Avenue",
            "Avene":"Avenue",
            "avenue":"Avenue",
            "Blvd":"Boulevard",
            "Ctr":"Center",
            "Dr":"Drive",
            "Plz":"Plaza",
            "Rd":"Road",
            "St":"Street",
            "St.":"Street",
            "ST":"Street"
            }

def update_name(name, mapping):
    '''Cleans name for insertion into database.'''
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            if street_type in mapping.keys():
                name = re.sub(street_type_re, mapping[street_type], name)
    return name



In [22]:
def test():
    st_types = audit_street(sample)
    #assert len(st_types) == 3
    #pprint.pprint(dict(st_types))
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
if __name__ == '__main__':
    test()

Bayside => Bayside
Grand Central Parkway Service Road West => Grand Central Parkway Service Road West
Prospect Park West => Prospect Park West
Columbia Heights => Columbia Heights
Union Turnpike => Union Turnpike
Avenue Of Puerto Rico => Avenue Of Puerto Rico
Avenue X => Avenue X
Village Road East => Village Road East
Cadman Plaza East => Cadman Plaza East
Williamsburg Street East => Williamsburg Street East
Gramercy Park East => Gramercy Park East
Bedford avenue => Bedford Avenue
Prospect Park Southwest => Prospect Park Southwest
Horace Harding Expressway => Horace Harding Expressway
Juniper Boulevard North => Juniper Boulevard North
Barlow Drive North => Barlow Drive North
Paerdegat Avenue North => Paerdegat Avenue North
Greenway North => Greenway North
Queens Midtown Expressway Service Road North => Queens Midtown Expressway Service Road North
Bowery => Bowery
Willoughby => Willoughby
Fort Hamilton => Fort Hamilton
Boelsen Crescent => Boelsen Crescent
Dieterle Crescent => Dieterle C

### "Incorrect" zipcodes

The zipcode of Brooklyn begin with “112”, we modified the functions before and check zipcode. From the result we can see most of the zip code format (5 digits in length) is correct, but there is a significant portion of zipcodes that don't belong in Brooklyn. For example, zipcodes like 07030 is from New Jersey and 10003 is from Manhattan.

In [23]:
def audit_zipcode(invalid_zipcodes, zipcode):
    '''Returns zipcodes that are not in the area in the osm file.'''
    threeDigits = zipcode[0:3]
    if threeDigits != 112 or not threeDigits.isdigit():
        invalid_zipcodes[threeDigits].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    '''Returns a dictionary of zipcodes in the osm file'''
    osm_file = open(osmfile, "r")
    invalid_zipcodes = collections.defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

bk_zipcode = audit_zip(sample)
pprint.pprint(dict(bk_zipcode))

{'070': set(['07030']),
 '073': set(['07302']),
 '100': set(['10002',
             '10003',
             '10004',
             '10005',
             '10006',
             '10007',
             '10009',
             '10010',
             '10011',
             '10012',
             '10013',
             '10014',
             '10038']),
 '102': set(['10282']),
 '111': set(['11101', '11104']),
 '112': set(['11201',
             '11203',
             '11204',
             '11205',
             '11206',
             '11207',
             '11208',
             '11209',
             '11210',
             '11211',
             '11212',
             '11213',
             '11214',
             '11215',
             '11215-9993',
             '11216',
             '11217',
             '11218',
             '11219',
             '11220',
             '11221',
             '11222',
             '11223',
             '11224',
             '11225',
             '11226',
             '11228',
        

In [24]:
def update_postcode(postcode):
    '''Returns 5 digit postcodes for insertion into SQL Database'''
    search = re.match(r'^\D*(\d{5}).*', postcode)
    clean_postcode = search.group(1)
    return clean_postcode

for st_type, ways in bk_zipcode.iteritems():
        for name in ways:
            better_name = update_postcode(name)
            print name, "=>", better_name

07302 => 07302
07030 => 07030
11414 => 11414
11415 => 11415
11416 => 11416
11417 => 11417
11418 => 11418
11419 => 11419
11421 => 11421
11697 => 11697
11694 => 11694
11101 => 11101
11104 => 11104
11379 => 11379
11385 => 11385
11368 => 11368
11375 => 11375
11374 => 11374
11377 => 11377
11373 => 11373
11367 => 11367
11378 => 11378
11207 => 11207
11228 => 11228
11201 => 11201
11203 => 11203
11205 => 11205
11204 => 11204
11229 => 11229
11226 => 11226
11209 => 11209
11208 => 11208
11225 => 11225
11224 => 11224
11223 => 11223
11218 => 11218
11221 => 11221
11220 => 11220
11219 => 11219
11249 => 11249
11215-9993 => 11215
11222 => 11222
11206 => 11206
11238 => 11238
11213 => 11213
11210 => 11210
11211 => 11211
11216 => 11216
11217 => 11217
11214 => 11214
11215 => 11215
11230 => 11230
11231 => 11231
11232 => 11232
11233 => 11233
11234 => 11234
11235 => 11235
11236 => 11236
11237 => 11237
11212 => 11212
11239 => 11239
10282 => 10282
10004 => 10004
10005 => 10005
10006 => 10006
10007 => 10007
10002

### "Incorrect" and Inconsistent phone numbers

Change the format of all phone numbers to (###) ###-####. Brooklyn phone area code is 718. Note that a portion of the phone numbers with the area code of 212 is from Manhattan and phone numbers with the area 201 is from New Jersey.

In [25]:
def is_phone(elem):
    '''Search elements that have phone numbers.'''
    return(elem.tag == 'tag') and (elem.attrib['k'] == "phone" or elem.atrrib['k']=='fax')

def phone_numbers(osmfile):
    '''Takes phone numbers from osm file and output a set of phone numbers'''
    osm_file = open(osmfile, "r")
    phone_nums = set()
    for event, elem in ET.iterparse(osmfile):
        if elem.tag in ["node" ,"way"]:
             for tag in elem.iter("tag"):
                if tag.attrib['k'] == "phone":
                    phone_nums.add(tag.attrib['v'])
    return phone_nums
                    
bk_phone = phone_numbers("sample.osm")
pprint.pprint(bk_phone)

set(['(212) 228-2004',
     '(212) 260-1175',
     '(212) 465-0880',
     '(212) 995-2020',
     '(718) 488-7005',
     '(718) 520-5444',
     '(718) 677-5811',
     '(718) 768-6868',
     '+1 212 254 1508',
     '+1 212 255-7260',
     '+1 212 691 1287',
     '+1 718 788-0050',
     '+1 844 359 2523',
     '+1-212-510-8551',
     '+1-718.436.0545',
     '+17188527800',
     '201 216-1766',
     '212-219-8787',
     '718 349 6020',
     '718 349 6555',
     '718 389 6965',
     '718) 235-0444',
     '718-418-0793',
     '7183993696'])


In [26]:
PHONENUM = re.compile(r'^\(\d{3}\)\s\d{3}-\d{4}$')

def update_phone(phone_num):
    '''Cleans phone number for insertion into SQL Database.'''
    if isinstance(phone_num, str):
        #Check for valid number format.
        m = PHONENUM.search(phone_num)
        if m is None:
            #Removes "+1-
            if phone_num.startswith('+1-'):
                phone_num = phone_num.replace('+1-','')
            #Removes +1
            if "+1" in phone_num:
                phone_num = phone_num.replace('+1','')
            #Removes '.' that were contained in phonenumbers
            if "." in phone_num:
                phone_num = phone_num.replace(".", "")
            #Removes dashes
            if "-" in phone_num:
                phone_num = re.sub("-", "", phone_num)
            #Removes brackets and parenthesis
            if "(" in phone_num or ")" in phone_num:
                phone_num = re.sub("[()]", "", phone_num)
            #Remove spaces inbetween digits of the phone number
            if " " in phone_num:
                phone_num = re.sub(" ", "", phone_num)
            #Format the phone numbers into (718) 123-4567 format.
            if re.match(r'\d{10}', phone_num) is not None:
                phone_num = "(" + phone_num[:3] + ")" + " "+ phone_num[3:6] + "-" + phone_num[6:]
        return phone_num
    
for phone_num in bk_phone:
    print update_phone(phone_num)


(718) 235-0444
(212) 260-1175
(718) 389-6965
(718) 488-7005
(718) 520-5444
(212) 254-1508
(212) 510-8551
(212) 228-2004
(212) 465-0880
(718) 852-7800
(212) 219-8787
(718) 349-6555
(718) 399-3696
(212) 691-1287
(718) 349-6020
(718) 418-0793
(844) 359-2523
(718) 768-6868
(718) 788-0050
(718) 436-0545
(718) 677-5811
(212) 255-7260
(201) 216-1766
(212) 995-2020


## Data Overview

In [27]:
import sqlite3
import csv
from pprint import pprint

In [28]:
sqlite_file = 'bklyn.db'    # name of the sqlite database file

# Connect to the database
conn = sqlite3.connect(sqlite_file)

In [29]:
# Get a cursor object
cur = conn.cursor()

### Number of Nodes

In [30]:
cur.execute('SELECT COUNT(*) FROM Nodes')
all_rows = cur.fetchall()
print(all_rows)

[(49947,)]


### Number of Ways

In [31]:
cur.execute('SELECT COUNT(*) FROM Ways')
all_rows = cur.fetchall()
print(all_rows)

[(9864,)]


### Number of Unique Users

In [32]:
cur.execute('SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;')
all_rows = cur.fetchall()
print(all_rows)

[(436,)]


### Top 10 Contributing Users

In [38]:
cur.execute('SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 10;')
top_ten = cur.fetchall()
pprint(top_ten)

[(u'Rub21_nycbuildings', 34778),
 (u'ingalls_nycbuildings', 7497),
 (u'ediyes_nycbuildings', 3786),
 (u'celosia_nycbuildings', 2337),
 (u'ingalls', 2118),
 (u'lxbarth_nycbuildings', 1606),
 (u'aaron_nycbuildings', 811),
 (u'ewedistrict_nycbuildings', 694),
 (u'smlevine', 501),
 (u'robgeb', 487)]


### Number of Unique Users

In [36]:
cur.execute("SELECT COUNT(DISTINCT(e.uid)) FROM \
          (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;")
print cur.fetchall()

[(436,)]


### Top 10 Amenities

In [40]:
cur.execute("SELECT value, COUNT(*) as num \
            FROM NodesTags \
           WHERE key='amenity' \
           GROUP BY value \
           ORDER BY num DESC \
           LIMIT 10;")

pprint(cur.fetchall())

[(u'bicycle_parking', 59),
 (u'restaurant', 15),
 (u'cafe', 10),
 (u'school', 10),
 (u'bar', 6),
 (u'bicycle_rental', 5),
 (u'fast_food', 5),
 (u'place_of_worship', 5),
 (u'bench', 4),
 (u'post_office', 3)]


In [43]:
cur.execute("SELECT NodesTags.value, COUNT(*) as num \
           FROM NodesTags \
               JOIN (SELECT DISTINCT(id) FROM NodesTags WHERE value = 'restaurant') \
               i ON NodesTags.id = i.id \
           WHERE NodesTags.key = 'cuisine'\
           GROUP BY NodesTags.value\
           ORDER BY num DESC;")

pprint(cur.fetchall())

[(u'pizza', 2),
 (u'Southern', 1),
 (u'diner', 1),
 (u'italian', 1),
 (u'ramen', 1),
 (u'soul_food', 1)]


# Conclusion

From the process of auditing we can see the dataset is not very well-cleaned even though there are alot of data outside of the brooklyn such as postal codes, phone numbers and street names. Since there are thousands of contributing users, so it is inevitable to have so many human input error. My thought is: is it possible to create a monitor system to check everybody’s contribution regularly. Another thought is to put a rough GPS data processor in place and working together with a more robust data processor similar to data.py. That way all the info of the area will be in the dataset. I think it would be possible to input a great amount of cleaned data to OpenStreetMap.